# Collaboration and Competition

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.5 which is incompatible.


The environment is already saved in the Workspace and can be accessed at the file path provided below. 

In [2]:
from unityagents import UnityEnvironment
import pandas as pd
import random
import torch

from config import Config
from normalizer import MeanStdNormalizer
from ppo_agent import PPOAgent
from network import GaussianActorCriticNet, FCBody

from workspace_utils import active_session

from utils import *
%matplotlib inline

RANDOM_SEED = 42

env = UnityEnvironment(file_name="/data/Tennis_Linux_NoVis/Tennis")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.         -6.65278625 -1.5        -0.          0.
  6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [5]:
for i in range(5):                                         # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to the environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.04500000085681677
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: -0.004999999888241291


When finished, you can close the environment.

In [6]:
# env.close()

### ppo_agent.py

#######################################################################
# Copyright (C) 2017 Shangtong Zhang(zhangshangtong.cpp@gmail.com)    #
# Permission given to modify the code as long as you keep this        #
# declaration at the top                                              #
#######################################################################

# Modified by Jeff Daniels (jeffrey.arthur.daniels@gmail.com) 2020
# For Udacity Deep Reinforcement Learning Nanodegree

import numpy as np
import torch
from replay import Storage
from torch_utils import to_np, tensor
from misc_utils import random_sample 

class PPOAgent():
    def __init__(self, config):
        self.config = config
        self.environment = config.environment
        self.brain_name = config.brain_name
        self.network = config.network_fn()
        self.actor_opt = config.actor_opt_fn(self.network.actor_params)
        self.critic_opt = config.critic_opt_fn(self.network.critic_params)
        self.total_steps = 0
        self.states = self.environment.reset(train_mode=True)[self.brain_name].vector_observations
        self.states = config.state_normalizer(self.states)

    def step(self):
        config = self.config
        storage = Storage(config.rollout_length)
        states = self.states
        
        for _ in range(config.rollout_length):
            prediction = self.network(states)

            environment_info = self.environment.step(to_np(prediction['a']))[self.brain_name]
            next_states = environment_info.vector_observations 
            rewards = environment_info.rewards
            terminals = environment_info.local_done
            terminals = np.array([1 if terminal else 0 for terminal in terminals])
            rewards = config.reward_normalizer(rewards)
            next_states = config.state_normalizer(next_states)
            storage.add(prediction)
            storage.add({'r': tensor(rewards).unsqueeze(-1),
                         'm': tensor(1 - terminals).unsqueeze(-1),
                         's': tensor(states)})
            states = next_states
            self.total_steps += config.num_agents
        
        self.states = states
        prediction = self.network(states)
        storage.add(prediction)
        storage.placeholder()

        advantages = tensor(np.zeros((config.num_workers, 1)))
        returns = prediction['v'].detach()
        for i in reversed(range(config.rollout_length)):
            returns = storage.r[i] + config.discount * storage.m[i] * returns
            if not config.use_gae:
                advantages = returns - storage.v[i].detach()
            else:
                td_error = storage.r[i] + config.discount * storage.m[i] * storage.v[i + 1] - storage.v[i]
                advantages = advantages * config.gae_tau * config.discount * storage.m[i] + td_error
            storage.adv[i] = advantages.detach()
            storage.ret[i] = returns.detach()

        states, actions, log_probs_old, returns, advantages = storage.cat(['s', 'a', 'log_pi_a', 'ret', 'adv'])
        actions = actions.detach()
        log_probs_old = log_probs_old.detach()
        advantages = (advantages - advantages.mean()) / advantages.std()

        for _ in range(config.optimization_epochs):
            sampler = random_sample(np.arange(states.size(0)), config.mini_batch_size)
            for batch_indices in sampler:
                batch_indices = tensor(batch_indices).long()
                sampled_states = states[batch_indices]
                sampled_actions = actions[batch_indices]
                sampled_log_probs_old = log_probs_old[batch_indices]
                sampled_returns = returns[batch_indices]
                sampled_advantages = advantages[batch_indices]

                prediction = self.network(sampled_states, sampled_actions)
                ratio = (prediction['log_pi_a'] - sampled_log_probs_old).exp()
                obj = ratio * sampled_advantages
                obj_clipped = ratio.clamp(1.0 - self.config.ppo_ratio_clip,
                                          1.0 + self.config.ppo_ratio_clip) * sampled_advantages
                policy_loss = -torch.min(obj, obj_clipped).mean() - config.entropy_weight * prediction['ent'].mean()

                value_loss = 0.5 * (sampled_returns - prediction['v']).pow(2).mean()

                approx_kl = (sampled_log_probs_old - prediction['log_pi_a']).mean()
                if approx_kl <= 1.5 * config.target_kl:
                    self.actor_opt.zero_grad()
                    policy_loss.backward()
                    self.actor_opt.step()

                self.critic_opt.zero_grad()
                value_loss.backward()
                self.critic_opt.step()
        


### utils.py

def save_checkpoint(agent, scores_per_episode, steps_per_episode, path='checkpoint.tar'):
    """ saves the model weights and optimizers, the scores_per_episode and the ReplayBuffer memory"""
    torch.save({'scores_per_episode': scores_per_episode, 
                'steps_per_episode': steps_per_episode,
                'network_dict': agent.network.state_dict(),
                'actor_opt_dict': agent.actor_opt.state_dict(),
                'critic_opt_dict': agent.critic_opt.state_dict()
              }, path)

def load_checkpoint(agents, scores_per_episode, steps_per_episode, path):
    """ loads a checkpoint to pick up training from the last saved checkpoint"""
    checkpoint = torch.load(path)
    scores_per_episode = checkpoint['scores_per_episode']
    steps_per_episode = checkpoint['steps_per_episode']
    agents.network.load_state_dict(checkpoint['network_dict'])
    agents.actor_opt.load_state_dict(checkpoint['actor_opt_dict'])
    agents.critic_opt.load_state_dict(checkpoint['critic_opt_dict'])
    
    return agents, scores_per_episode, steps_per_episode

def play_episode(agent):
    """ 
    Plays a single episode of the environment
    returns the mean score of all the agents, and the combined steps of the agents
    """
    config = agent.config
    env = config.environment
    brain_name = config.brain_name
    env_info = env.reset(train_mode=True)[brain_name]    
    states = env_info.vector_observations                 
    scores = np.zeros(config.num_agents)
    steps = 0
    while True:
        prediction = agent.network(states)
        env_info = env.step(prediction['a'].cpu().detach().numpy())[brain_name]
        next_states = env_info.vector_observations         
        rewards = env_info.rewards                         
        dones = env_info.local_done                     
        scores += rewards                      
        states = next_states      
        steps += config.num_agents
        if np.any(dones):                                  
            break
    
    return scores, steps

import time 

def run_steps(agent):
    config = agent.config
    scores_per_episode = []
    steps_per_episode = []
    max_scores_window = deque(maxlen=config.scores_window_len)
    steps_window = deque(maxlen=config.scores_window_len)
    toc = time.time()
    
    if config.load_checkpoint:
        agent, scores_per_episode, steps_per_episode = \
        load_checkpoint(agent, scores_per_episode,
                        steps_per_episode, config.load_checkpoint_path)
        for score in scores_per_episode:
            max_scores_window.append(score.max())
        for step in steps_per_episode:
            steps_window.append(step)
    
    for episode in range(len(scores_per_episode)+1, config.max_episodes+1):
        agent.step()
        score, steps = play_episode(agent)
        scores_per_episode.append(score)
        max_scores_window.append(score.max())
        steps_per_episode.append(steps)
        steps_window.append(steps)
        
        # Print out scores after each episode and every print interval
        tic = time.time()
        s = '\rEpisode: {}\tScores: {:.3f} {:.3f}\tSteps: {}\tTime Elapsed: {:.1f} min'
        print(s.format(episode, score[0], score[1], steps, (tic-toc)/60), end=" ")
        
        if episode % config.print_interval == 0:                
            s = '\rEpisode: {}\tAverage Max Score: {:.3f}\tAverage steps: {:.1f}'
            print(s.format(episode, np.mean(max_scores_window), 
                           np.mean(steps_per_episode)), end=" "*35+"\n")
        
        if episode % config.save_interval == 0:
            path = f'checkpoint-{episode}.tar'
            save_checkpoint(agent, scores_per_episode, steps_per_episode, path)
        
        if (np.mean(max_scores_window) >= config.target_score and 
            episode >= config.scores_window_len):
            print('\nEnvironment solved in {} episodes!!!'.format(episode))
            print('Average score over {} episodes was: {:.3f}'
                  .format(config.scores_window_len, np.mean(max_scores_window)))
            print('Time Elapsed: {:.1f} min'.format((tic-toc)/60))
            path = f'checkpoint-{episode}.tar'
            save_checkpoint(agent, scores_per_episode, steps_per_episode, path)
            break
        
    print('Time Elapsed: {:.1f} min'.format((tic-toc)/60))
    
    return (agent, scores_per_episode, steps_per_episode)

        

# `RUN ALL ABOVE`

### PPO Example

In [41]:
def ppo_continuous(**kwargs):
    kwargs.setdefault('log_level', 0)
    config = Config()
    config.merge(kwargs)
    config.environment = env
    config.brain_name = brain_name
    config.state_dim = state_size
    config.action_dim = action_size
    config.num_agents = num_agents
    
    # Hyperparameters
    config.network_fn = lambda: GaussianActorCriticNet(
        config.state_dim, config.action_dim, 
        actor_body=FCBody(config.state_dim, hidden_units=(128, 128), gate=torch.tanh),
        critic_body=FCBody(config.state_dim, hidden_units=(128, 128), gate=torch.tanh))
    config.actor_opt_fn = lambda params: torch.optim.Adam(params, 3e-4)
    config.critic_opt_fn = lambda params: torch.optim.Adam(params, 1e-3)
    config.discount = 0.99
    config.use_gae = True
    config.gae_tau = 0.95
    config.gradient_clip = 0.5    
    config.rollout_length = 256
    config.optimization_epochs = 10
    config.mini_batch_size = 32 
    config.ppo_ratio_clip = 0.2
    config.log_interval = 2048
    config.max_steps = int(1e4)
    config.target_kl = 0.01
    config.state_normalizer = MeanStdNormalizer()
    
    # Environment solving criteria
    config.scores_window_len = 100
    config.target_score = 0.5
        
    # Training Parameters
    config.max_episodes = 20
    config.print_interval = 10
    config.save_interval = 100
    config.load_checkpoint = False
    config.load_checkpoint_path = f'checkpoint-{10}.tar'
    ppo_agent = PPOAgent(config)
    agent, scores_per_episode, steps_per_episode = run_steps(ppo_agent)
    return agent, scores_per_episode, steps_per_episode



#!----------------TRAINING BELOW-------------------!#
'''
active_session() disallows workspace slumbering
https://github.com/udacity/workspaces-student-support/tree/master/jupyter
'''

with active_session():
    agent, scores_per_episode, steps_per_episode = ppo_continuous()

Episode: 10	Average Max Score: 0.039	Average steps: 38.4                                   
Episode: 20	Average Max Score: 0.030	Average steps: 31.3                                   
Episode: 30	Average Max Score: 0.039	Average steps: 33.9                                   
Episode: 40	Average Max Score: 0.041	Average steps: 34.0                                   
Episode: 50	Average Max Score: 0.042	Average steps: 32.8                                   
Episode: 60	Average Max Score: 0.043	Average steps: 32.2                                   
Episode: 70	Average Max Score: 0.040	Average steps: 30.0                                   
Episode: 80	Average Max Score: 0.042	Average steps: 30.5                                   
Episode: 90	Average Max Score: 0.045	Average steps: 31.9                                   
Episode: 100	Average Max Score: 0.044	Average steps: 31.9                                   
Time Elapsed: 1.6 min


# `Set Config and Run All above`

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 